# Monotonicity Violations

In [1]:
from pref_voting.generate_profiles import *
from pref_voting.profiles import *
from pref_voting.voting_methods import *
from pref_voting.analysis import *
from functools import partial
from multiprocess import Pool, cpu_count
import pandas as pd
from tqdm.notebook import tqdm


In [2]:
## The following functions are available in pref_voting.analysis, but are provided here for reference. 
#

def simple_lift(ranking, c):
    """
    Return a ranking in which ``c`` is moved up one position in ``ranking``.
    """
    assert c != ranking[0], "can't lift a candidate already in first place"
    
    new_ranking = copy.deepcopy(ranking)
    c_idx = new_ranking.index(c)
    new_ranking[c_idx - 1], new_ranking[c_idx] = new_ranking[c_idx], new_ranking[c_idx-1]
    return new_ranking

def has_monotonicity_violation(profile, vm, verbose = False): 
    
    _rankings, _rcounts = profile.rankings_counts

    rankings = [list(r) for r in list(_rankings)]
    rcounts = list(_rcounts)
    old_rankings = copy.deepcopy(rankings)

    ws = vm(profile)
    for w in ws: 
        for r_idx, r in enumerate(rankings): 
            if r[0] != w:
                old_ranking = copy.deepcopy(r)
                new_ranking = simple_lift(r, w)
                new_rankings = old_rankings + [new_ranking]
                new_rcounts  = copy.deepcopy(rcounts + [1])
                new_rcounts[r_idx] -= 1
                new_prof = Profile(new_rankings, new_rcounts)
                new_ws = vm(new_prof)
                if w not in new_ws: 
                    if verbose: 
                        print(f"monotonicity violation for {vm.name}")
                        profile.display()
                        print(f"{vm.name} winners: ", ws)
                        print("original ranking ", old_ranking)
                        print(f"new ranking {new_ranking}")
                        new_prof.display()
                        print(f"{vm.name} winners in updated profile ", new_ws)
                    return True
    return False


In [3]:
def record_mon_data(num_candidates, num_voters, vms, t): 
    
    prof = generate_profile(num_candidates, num_voters)
    
    return {vm.name: has_monotonicity_violation(prof, vm) for vm in vms}

In [4]:
all_num_cands = [
#     4, 
#     5, 
    6
]

all_num_voters = [
    4, 
    5, 
    10, 
    11, 
    20, 
    21,
    50, 
    51
]

vms = [
    instant_runoff_put,
    smith_irv_put,
    stable_voting
]

num_trials = 100

# use parallel processing to speed up the search
cpus = cpu_count()
print(f'CPUS: {cpus}')
pool = Pool(12) # set to cpus when you can use all the processors



CPUS: 16


In [ ]:
%%time

data_for_df = {
    "num_cands": list(),
    "num_voters": list(),
    "num_profiles": list(),
    "vm": list(),
    "perc_monotonicity_violation": list(),
}
for num_candidates in all_num_cands: 
    print(f"{num_candidates=}")
    for num_voters in all_num_voters:
        print(f"{num_voters=}")
        
        record_violations = partial(
            record_mon_data,
            num_candidates, 
            num_voters,
            vms
        )

        violations = pool.map(record_violations, range(num_trials))

        for vm in vms: 
            data_for_df["num_cands"].append(num_candidates)
            data_for_df["num_voters"].append(num_voters)
            data_for_df["vm"].append(vm.name)
            data_for_df["num_profiles"].append(len(violations))
            data_for_df["perc_monotonicity_violation"].append(sum([d[vm.name] for d in violations]) / len(violations))
        #data = [record_mon_data(num_candidates, num_voters, vms, t) for t in tqdm(range(num_trials))]
        df = pd.DataFrame(data_for_df)
        df.to_csv("./monotoncity_violations.csv", index=False)
        #print(data)

num_candidates=6
num_voters=4
num_voters=5
num_voters=10
num_voters=11


In [6]:
df

,num_cands,num_voters,num_profiles,vm,perc_monotonicity_violation
0,6,4,100,Instant Runoff PUT,0.00
1,6,4,100,Smith IRV PUT,0.00
2,6,4,100,Stable Voting,0.00
3,6,5,100,Instant Runoff PUT,0.00
4,6,5,100,Smith IRV PUT,0.00
5,6,5,100,Stable Voting,0.00
6,6,10,100,Instant Runoff PUT,0.00
7,6,10,100,Smith IRV PUT,0.00
8,6,10,100,Stable Voting,0.00
9,6,11,100,Instant Runoff PUT,0.10
